Read US Cities CSV file and put each city/state into a list

In [4]:
import csv 

filename = 'uscities.csv'
cities = []

with open(filename,'r') as file:
    reader = csv.reader(file)
    next(reader)
    for line in reader:
        city = line[0].replace(' ','-')
        city = city.replace('St.','Saint')
        state = line[2]
        cities.append((city,state))
        

Use the list of cities and states to webscrape the cost of living data and add it to the cost_of_living.csv

In [ ]:
import requests
import sys
from bs4 import BeautifulSoup
import csv

cost_living_csv = 'cost_of_living.csv'
cities_added = 'cities_add.csv'
header_made = 0
cities = cities[1350:]

#Filter through the html to get the table data, then add it to the csv
def add_city_data(response,city_name,state):
    global header_made
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    table = soup.find('table', class_='data_wide_table')
    if not table: 
        return #end if table does not exist

    row_data = [city_name,state]
    header = ['City','State']

    for row in table.find_all('tr'):
        cols = row.find_all('td')
        data = [td.get_text(strip=True) for td in cols]
        if len(data) > 2:
            item = data[0].strip()
            avg_price = data[1].replace('\xa0$','').strip()
            price_range = data[2].strip()
            prices = (avg_price,price_range)
            row_data.append(prices)
            header.append(item)

    with open(cost_living_csv, 'a', newline = '') as file:
        writer = csv.writer(file)
        if not header_made:
            writer.writerow(header)
            header_made = 1
        writer.writerow(row_data)
        file.flush()

#Create URL from the city/state and retreive website response, call add_city_data function with response as an input
def fetch_data(city,state):
    city_name = city.replace('-',' ')
    url = f'https://www.numbeo.com/cost-of-living/in/{city}'
    backup_url = f'https://www.numbeo.com/cost-of-living/in/{city}-{state}-United-States'
    urls = [url,backup_url]

    for url in urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                add_city_data(response,city_name,state)
        except:
                sys.stderr.write(f'Cannot retreive data for {city_name}\n')

for city, state in cities:
    fetch_data(city,state)

Webscrape Quality of Life data and add to quality_of_life.csv 

In [ ]:
import requests
import sys
from bs4 import BeautifulSoup
import csv

quality_life_csv = 'quality_of_life.csv'
header_made = 0
# cities = cities[101:]

#Filter through the html to get the table data, then add it to the csv
def add_quality_life_data(response,city_name,state):
    global header_made
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    tables = soup.find_all('table')
    table = tables[2]
    if not table: 
        return #end if table does not exist

    row_data = [city_name,state]
    header = ['City','State']

    for row in table.find_all('tr'):
        cols = row.find_all('td')
        data = [td.get_text(strip=True) for td in cols]
        if len(data) > 2:
            item = data[0].strip()
            quality = data[1].strip()
            rating = data[2].strip()
            qualities = (quality,rating)
            row_data.append(qualities)
            header.append(item)

    with open(quality_life_csv, 'a', newline = '') as file:
        writer = csv.writer(file)
        if not header_made:
            writer.writerow(header)
            header_made = 1
        writer.writerow(row_data)
        file.flush()
        print(f'{city_name} added')

#Create URL from the city/state and retreive website response, call add_quality_life_data function with response as an input
def fetch_quality_life_data(city,state):
    city_name = city.replace('-',' ')
    url = f'https://www.numbeo.com/quality-of-life/in/{city}'
    backup_url = f'https://www.numbeo.com/quality-of-life/in/{city}-{state}-United-States'
    urls = [url,backup_url]

    for url in urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                add_quality_life_data(response,city_name,state)
                break
        except:
                sys.stderr.write(f'Cannot retreive data for {city_name}\n')

for city, state in cities:
    fetch_quality_life_data(city,state)

New York added
Los Angeles added
Chicago added
Miami added
Houston added


Get Average Salary for non-farm jobs in a particular city/area code. Use the Bureau of Labor statistics API

In [ ]:
import json
import requests
import sys

#file names for state and area codes
area_codes_file = 'area_codes.txt'
state_codes_file = 'state_codes.txt'
industry_codes_file = 'industry_codes.txt'
state_abbrevs_file = 'state_abbreviations.txt'

#api key and endpoint for U.S. Bureau of Labor Statistics
api_key = 'abbbbae8905543da8a40822dc27e7b88'
endpoint = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

#initialize dictionaries
area_code_dict = dict()
state_code_dict = dict()
industry_code_dict = dict()
state_abbrevs_dict = dict()

def get_BLS_data(seriesId, **kwargs):
    #Pass BLS timeseries data to return data in JSON format, arguments can be added to specify search
    if len(seriesId) < 1 or len(seriesId) > 25:
        raise ValueError('SeriesId must be at least 1 and less than 25')
    
    headers = {'Content-Type': 'application/json'}
    payload = {
        'seriesid': [seriesId],
        'registrationKey': api_key
    }
    
    #Update payload with keyword arguments and convert to JSON
    payload.update(kwargs)
    payload = json.dumps(payload)

    #Post request to BLS API
    response = requests.post(endpoint, data = payload, headers = headers)
    response.raise_for_status()

    #Return JSON response
    result = json.loads(response.text)
    if result['status'] != 'REQUEST_SUCCEEDED':
        raise Exception(result['message'])
    if result['status'] == 'REQUEST_SUCCEEDED':
        return result
    else:
        return None


Read from the given files that include area codes, state codes, industry codes, and state abbreviations, and convert into a dictionary

In [ ]:
with open(area_codes_file, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        code,city_state = line.strip().split('\t')
        area_code_dict[code] = city_state

with open(state_codes_file,'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        code,state = line.strip().split('\t')
        state_code_dict[state.strip()] = code

with open(state_abbrevs_file,'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        code,state = line.strip().split('\t')
        state_abbrevs_dict[state.strip()] = code

with open(industry_codes_file,'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        code,industry = line.strip().split('\t')
        industry_code_dict[code] = industry.strip()


In [ ]:
import csv 

bls_data_file = 'bls_data.csv'

def add_BLS_data(average_salary,area_code,state_name,abbrev):
    with open(bls_data_file,'a',newline='') as file:
        writer = csv.writer(file)
        row = [area_code,state_name,abbrev,average_salary]
        writer.writerow(row)
        file.flush()
        print(f'{row} added')

#Build the Series IDs for every zip code
series_ids = []
n = 0
failures = 0
industry_code = '00000000'

for area_code,city_state in area_code_dict.items():
    state_abbrev = city_state.split(',')[1].split('-')
    state_codes = []
    for abbrev in state_abbrev:
        abbrev = abbrev.strip()
        state_name = state_abbrevs_dict.get(abbrev[:2])
        state_code = state_code_dict.get(state_name)
        
        seriesId = f'SMU{state_code}{area_code}{industry_code}01'
        try:
            data = get_BLS_data(seriesId, startyear = 2024, endyear = 2024)['Results']['series'][0]['data']
            average_salary = data[0]['value']
            if average_salary:
                add_BLS_data(average_salary,area_code,state_name,abbrev)
                print(f'{average_salary} added')
        except:
            failures += 1
            print(f'{seriesId} fail')
            
        n += 1

# json_data = get_BLS_data(seriesId, startyear = 2023, endyear = 2024)
print(f'n: {n}\n failures: {failures}')
